In [152]:
from ferry_portfolio_pipeline.config.settings import CONFIG
import boto3
import io
import pandas as pd
from io import StringIO
import matplotlib.pyplot as plt
import joblib

bucket = CONFIG["s3"]["bucket"]
input_key = CONFIG["s3"]["key"]
output_key = CONFIG["s3"]["output_key"]
model_key = CONFIG["s3"]["model_key"]

s3 = boto3.client("s3")
obj = s3.get_object(Bucket=bucket, Key=output_key)
data = obj['Body'].read().decode('utf-8')
df = pd.read_csv(StringIO(data))

obj = s3.get_object(Bucket=bucket, Key= model_key)
model = joblib.load(io.BytesIO(obj['Body'].read()))



In [153]:
df['Date'] = pd.to_datetime(df['Date'])
df_date = df.groupby(['Date']).sum()[['Sales Count','Redemption Count']]
df = df_date.resample('ME').sum()[['Sales Count']].reset_index()
df = df[df['Date'] >= '2023-01-01']


In [154]:
future = model.make_future_dataframe(periods=len(df), freq='M')
forecast = model.predict(future)

import pandas as pd

# Get first day of next month
start_date = (pd.Timestamp.today().replace(day=1) + pd.DateOffset(months=1))

# End date is 4 months after start_date
end_date = start_date + pd.DateOffset(months=10)

# Filter forecast
forecast = forecast[(forecast['ds'] >= start_date) & (forecast['ds'] <= end_date)]



g:\DATASCIENCE\End_to_End_Project\DataAnalysis\Ferry_Forecasting\ferry\Lib\site-packages\prophet\forecaster.py:1872: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



In [155]:
import pandas as pd
import plotly.graph_objects as go

# Create the figure
fig = go.Figure()

# Plot actual sales with smooth line
fig.add_trace(go.Scatter(
    x=df['Date'],
    y=df['Sales Count'],
    mode='lines+markers+text',
    text=df['Sales Count'],
    textposition='top center',
    textfont=dict(size=9, color='blue'),
    name='Actual Sales',
    line=dict(color='blue', width=2, shape='spline', smoothing=1.3),
    marker=dict(size=6, color='blue')
))

# Create a seamless connection by starting forecast from the last actual data point
last_actual_date = df['Date'].iloc[-1]
last_actual_value = df['Sales Count'].iloc[-1]

# Combine last actual point with forecast for smooth transition
forecast_dates = [last_actual_date] + list(forecast['ds'])
forecast_values = [last_actual_value] + list(forecast['yhat'])

# Plot forecast sales with dotted line
fig.add_trace(go.Scatter(
    x=forecast_dates,
    y=forecast_values,
    mode='lines+markers+text',
    text=[''] + [f"{int(val)}" for val in forecast['yhat']],  # Empty text for connecting point
    textposition='top center',
    textfont=dict(size=9, color='red'),
    name='Forecast',
    line=dict(color='red', width=2, dash='dot', shape='spline', smoothing=1.3),
    marker=dict(size=6, color='red')
))

# Combine all dates for x-axis ticks
all_dates = list(df['Date']) + list(forecast['ds'])

# Update layout for better appearance
fig.update_layout(
    title='Ferry Sales Forecast',
    xaxis_title='Date',
    yaxis_title='Sales Count',
    template='plotly_white',
    height=500,  # Increase height to give more space for labels
    margin=dict(t=80, b=60, l=60, r=60),  # Add margins for text labels
    xaxis=dict(
        tickmode='array',
        tickvals=all_dates,  
        ticktext=[d.strftime('%b %Y') for d in all_dates],
        tickangle=45
    ),
    yaxis=dict(
        gridcolor='lightgray',
        gridwidth=0.5
    ),
    legend=dict(
        x=0.02,
        y=0.98,
        bgcolor='rgba(255,255,255,0.8)',
        bordercolor='gray',
        borderwidth=1
    ),
    hovermode='x unified'
)

# Add a subtle background grid
fig.update_yaxes(showgrid=True, gridwidth=0.5, gridcolor='lightgray')
fig.update_xaxes(showgrid=True, gridwidth=0.5, gridcolor='lightgray')

fig.show()

In [156]:
# Revenue Plotting Code
import plotly.graph_objects as go

# Set ticket price
ticket_price = 8.0  # CAD

# Calculate historical revenue
df_revenue = df.copy()
df_revenue['Revenue'] = df_revenue['Sales Count'] * ticket_price

# Calculate forecasted revenue
forecast_revenue = forecast.copy()
forecast_revenue['Revenue'] = forecast_revenue['yhat'] * ticket_price

# Create revenue forecast plot
fig = go.Figure()

# Plot actual revenue
fig.add_trace(go.Scatter(
    x=df_revenue['Date'],
    y=df_revenue['Revenue'],
    mode='lines+markers+text',
    text=[f"${int(val):,}" for val in df_revenue['Revenue']],
    textposition='top center',
    textfont=dict(size=9, color='blue'),
    name='Actual Revenue',
    line=dict(color='blue', width=2, shape='spline', smoothing=1.3),
    marker=dict(size=6, color='blue')
))

# Create seamless connection for forecast
last_actual_date = df_revenue['Date'].iloc[-1]
last_actual_value = df_revenue['Revenue'].iloc[-1]

forecast_dates = [last_actual_date] + list(forecast_revenue['ds'])
forecast_values = [last_actual_value] + list(forecast_revenue['Revenue'])

# Plot forecast revenue
fig.add_trace(go.Scatter(
    x=forecast_dates,
    y=forecast_values,
    mode='lines+markers+text',
    text=[''] + [f"${int(val):,}" for val in forecast_revenue['Revenue']],
    textposition='top center',
    textfont=dict(size=9, color='red'),
    name='Forecast Revenue',
    line=dict(color='red', width=2, dash='dot', shape='spline', smoothing=1.3),
    marker=dict(size=6, color='red')
))

# Combine all dates for x-axis
all_dates = list(df_revenue['Date']) + list(forecast_revenue['ds'])

# Update layout
fig.update_layout(
    title='Ferry Revenue Forecast - Monthly Data (Ticket Price: $8 CAD)',
    xaxis_title='Date',
    yaxis_title='Revenue (CAD)',
    template='plotly_white',
    height=500,
    margin=dict(t=80, b=60, l=60, r=60),
    xaxis=dict(
        tickmode='array',
        tickvals=all_dates,
        ticktext=[d.strftime('%b %Y') for d in all_dates],
        tickangle=45,
        showgrid=True,
        gridcolor='lightgray',
        gridwidth=0.5
    ),
    yaxis=dict(
        showgrid=True,
        gridcolor='lightgray',
        gridwidth=0.5,
        tickformat='$,.0f'  # Format as currency
    ),
    legend=dict(
        x=0.02,
        y=0.98,
        bgcolor='rgba(255,255,255,0.8)',
        bordercolor='gray',
        borderwidth=1
    ),
    hovermode='x unified'
)

fig.show()

In [157]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
from datetime import datetime

# Set ticket price
ticket_price = 8.0

# Calculate revenue for historical data
df_revenue = df.copy()
df_revenue['Revenue'] = df_revenue['Sales Count'] * ticket_price

# Calculate revenue for forecast data
forecast_revenue = forecast.copy()
forecast_revenue['Revenue'] = forecast_revenue['yhat'] * ticket_price

# Get current date info
current_date = pd.Timestamp.today()
current_month = current_date.replace(day=1)
next_month = current_month + pd.DateOffset(months=1)

# Extract KPI values
def get_kpis():
    kpis = {}
    
    # This month actual (if available)
    this_month_actual = df_revenue[df_revenue['Date'] == current_month]
    if not this_month_actual.empty:
        kpis['this_month_sales'] = this_month_actual['Sales Count'].iloc[0]
        kpis['this_month_revenue'] = this_month_actual['Revenue'].iloc[0]
    else:
        # Use last available month as "current"
        last_month = df_revenue['Date'].max()
        last_data = df_revenue[df_revenue['Date'] == last_month]
        kpis['this_month_sales'] = last_data['Sales Count'].iloc[0]
        kpis['this_month_revenue'] = last_data['Revenue'].iloc[0]
        kpis['this_month_label'] = f"Latest ({last_month.strftime('%b %Y')})"
    
    # Next month forecast
    next_month_forecast = forecast_revenue.iloc[0]  # First forecast point
    kpis['next_month_sales'] = int(next_month_forecast['yhat'])
    kpis['next_month_revenue'] = next_month_forecast['Revenue']
    
    # Growth calculations
    if len(df_revenue) >= 2:
        prev_month = df_revenue.iloc[-2]
        current_month_data = df_revenue.iloc[-1]
        
        # Store previous month values for delta reference
        kpis['prev_month_sales'] = prev_month['Sales Count']
        kpis['prev_month_revenue'] = prev_month['Revenue']
        
        # Calculate growth percentages
        kpis['sales_growth'] = ((current_month_data['Sales Count'] - prev_month['Sales Count']) / prev_month['Sales Count']) * 100
        kpis['revenue_growth'] = ((current_month_data['Revenue'] - prev_month['Revenue']) / prev_month['Revenue']) * 100
    else:
        # Fallback if not enough data
        kpis['prev_month_sales'] = kpis['this_month_sales']
        kpis['prev_month_revenue'] = kpis['this_month_revenue']
        kpis['sales_growth'] = 0
        kpis['revenue_growth'] = 0
    
    # Forecast vs current comparison
    kpis['forecast_sales_change'] = ((kpis['next_month_sales'] - kpis['this_month_sales']) / kpis['this_month_sales']) * 100
    kpis['forecast_revenue_change'] = ((kpis['next_month_revenue'] - kpis['this_month_revenue']) / kpis['this_month_revenue']) * 100
    
    return kpis

# Get KPI values
kpis = get_kpis()

# Create KPI Dashboard
fig = make_subplots(
    rows=2, cols=3,
    subplot_titles=[
        "Current Month Sales", "Next Month Forecast Sales", "Sales Growth",
        "Current Month Revenue", "Next Month Forecast Revenue", "Revenue Growth"
    ],
    specs=[[{"type": "indicator"}, {"type": "indicator"}, {"type": "indicator"}],
           [{"type": "indicator"}, {"type": "indicator"}, {"type": "indicator"}]]
)

# Current Month Sales KPI
fig.add_trace(go.Indicator(
    mode="number+delta",
    value=kpis['this_month_sales'],
    number={'suffix': " tickets", 'font': {'size': 30}},
    delta={'reference': kpis.get('prev_month_sales', kpis['this_month_sales']), 'relative': True, 'position': "top"},
    title={"text": kpis.get('this_month_label', 'Current Month')},
    domain={'row': 0, 'column': 0}
), row=1, col=1)

# Next Month Forecast Sales KPI
fig.add_trace(go.Indicator(
    mode="number+delta",
    value=kpis['next_month_sales'],
    number={'suffix': " tickets", 'font': {'size': 30}},
    delta={'reference': kpis['this_month_sales'], 'relative': True, 'position': "top"},
    title={"text": "Next Month Forecast"},
    domain={'row': 0, 'column': 1}
), row=1, col=2)

# Sales Growth KPI
fig.add_trace(go.Indicator(
    mode="number+gauge",
    value=kpis.get('sales_growth', 0),
    number={'suffix': "%", 'font': {'size': 30}},
    gauge={'axis': {'range': [-50, 50]},
           'bar': {'color': "green" if kpis.get('sales_growth', 0) > 0 else "red"},
           'steps': [{'range': [-50, 0], 'color': "lightgray"},
                    {'range': [0, 50], 'color': "lightgreen"}],
           'threshold': {'line': {'color': "red", 'width': 4},
                        'thickness': 0.75, 'value': 0}},
    title={"text": "Month-over-Month Sales Growth"},
    domain={'row': 0, 'column': 2}
), row=1, col=3)

# Current Month Revenue KPI
fig.add_trace(go.Indicator(
    mode="number+delta",
    value=kpis['this_month_revenue'],
    number={'prefix': "$", 'font': {'size': 30}, 'valueformat': ',.0f'},
    delta={'reference': kpis.get('prev_month_revenue', kpis['this_month_revenue']), 'relative': True, 'position': "top"},
    title={"text": kpis.get('this_month_label', 'Current Month Revenue')},
    domain={'row': 1, 'column': 0}
), row=2, col=1)

# Next Month Forecast Revenue KPI
fig.add_trace(go.Indicator(
    mode="number+delta",
    value=kpis['next_month_revenue'],
    number={'prefix': "$", 'font': {'size': 30}, 'valueformat': ',.0f'},
    delta={'reference': kpis['this_month_revenue'], 'relative': True, 'position': "top"},
    title={"text": "Next Month Forecast Revenue"},
    domain={'row': 1, 'column': 1}
), row=2, col=2)

# Revenue Growth KPI
fig.add_trace(go.Indicator(
    mode="number+gauge",
    value=kpis.get('revenue_growth', 0),
    number={'suffix': "%", 'font': {'size': 30}},
    gauge={'axis': {'range': [-50, 50]},
           'bar': {'color': "green" if kpis.get('revenue_growth', 0) > 0 else "red"},
           'steps': [{'range': [-50, 0], 'color': "lightgray"},
                    {'range': [0, 50], 'color': "lightgreen"}],
           'threshold': {'line': {'color': "red", 'width': 4},
                        'thickness': 0.75, 'value': 0}},
    title={"text": "Month-over-Month Revenue Growth"},
    domain={'row': 1, 'column': 2}
), row=2, col=3)

# Update layout
fig.update_layout(
    title={
        'text': 'Ferry Sales & Revenue KPI Dashboard',
        'x': 0.5,
        'font': {'size': 24, 'family': 'Arial Black'}
    },
    height=600,
    margin=dict(t=100, b=50, l=50, r=50),
    paper_bgcolor='white',
    font=dict(family="Arial", size=12)
)

fig.show()

# Print detailed KPI summary
print("\n" + "="*70)
print("FERRY OPERATIONS - EXECUTIVE KPI SUMMARY")
print("="*70)
print(f"📊 CURRENT PERFORMANCE:")
print(f"   Current Month Sales: {kpis['this_month_sales']:,} tickets")
print(f"   Current Month Revenue: ${kpis['this_month_revenue']:,.0f} CAD")
print(f"   Month-over-Month Growth: {kpis.get('sales_growth', 0):+.1f}%")
print()
print(f"🔮 NEXT MONTH FORECAST:")
print(f"   Forecasted Sales: {kpis['next_month_sales']:,} tickets")
print(f"   Forecasted Revenue: ${kpis['next_month_revenue']:,.0f} CAD")
print(f"   Expected Change: {kpis['forecast_sales_change']:+.1f}% sales, {kpis['forecast_revenue_change']:+.1f}% revenue")
print()
print(f"💰 KEY METRICS:")
print(f"   Average Ticket Price: ${ticket_price:.2f} CAD")
print(f"   Revenue per Ticket: ${kpis['this_month_revenue']/kpis['this_month_sales']:.2f} CAD")
print("="*70)


FERRY OPERATIONS - EXECUTIVE KPI SUMMARY
📊 CURRENT PERFORMANCE:
   Current Month Sales: 199,215 tickets
   Current Month Revenue: $1,593,720 CAD
   Month-over-Month Growth: -46.2%

🔮 NEXT MONTH FORECAST:
   Forecasted Sales: 211,682 tickets
   Forecasted Revenue: $1,693,459 CAD
   Expected Change: +6.3% sales, +6.3% revenue

💰 KEY METRICS:
   Average Ticket Price: $8.00 CAD
   Revenue per Ticket: $8.00 CAD


In [166]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

# Set ticket price
ticket_price = 8.0

# Calculate revenue
df_revenue = df.copy()
df_revenue['Revenue'] = df_revenue['Sales Count'] * ticket_price

forecast_revenue = forecast.copy()
forecast_revenue['Revenue'] = forecast_revenue['yhat'] * ticket_price

# Get KPI values (your original function)
def get_kpis():
    kpis = {}
    last_month = df_revenue['Date'].max()
    last_data = df_revenue[df_revenue['Date'] == last_month]
    kpis['this_month_sales'] = last_data['Sales Count'].iloc[0]
    kpis['this_month_revenue'] = last_data['Revenue'].iloc[0]
    kpis['this_month_label'] = f"Latest ({last_month.strftime('%b %Y')})"
    
    next_month_forecast = forecast_revenue.iloc[0]
    kpis['next_month_sales'] = int(next_month_forecast['yhat'])
    kpis['next_month_revenue'] = next_month_forecast['Revenue']
    
    if len(df_revenue) >= 2:
        prev_month = df_revenue.iloc[-2]
        current_month_data = df_revenue.iloc[-1]
        kpis['prev_month_sales'] = prev_month['Sales Count']
        kpis['prev_month_revenue'] = prev_month['Revenue']
        kpis['sales_growth'] = ((current_month_data['Sales Count'] - prev_month['Sales Count']) / prev_month['Sales Count']) * 100
        kpis['revenue_growth'] = ((current_month_data['Revenue'] - prev_month['Revenue']) / prev_month['Revenue']) * 100
    else:
        kpis['prev_month_sales'] = kpis['this_month_sales']
        kpis['prev_month_revenue'] = kpis['this_month_revenue']
        kpis['sales_growth'] = 0
        kpis['revenue_growth'] = 0
    
    kpis['forecast_sales_change'] = ((kpis['next_month_sales'] - kpis['this_month_sales']) / kpis['this_month_sales']) * 100
    kpis['forecast_revenue_change'] = ((kpis['next_month_revenue'] - kpis['this_month_revenue']) / kpis['this_month_revenue']) * 100
    
    return kpis

kpis = get_kpis()

# Create layout with KPIs at top, then your charts
fig = make_subplots(
    rows=4, cols=3,
    specs=[
        # Row 1: All 6 KPIs (your complete KPI dashboard)
        [{"type": "indicator"}, {"type": "indicator"}, {"type": "indicator"}],
        # Row 2: Second row of KPIs
        [{"type": "indicator"}, {"type": "indicator"}, {"type": "indicator"}],
        # Row 3: Sales forecast chart (full width)
        [{"colspan": 3}, None, None],
        # Row 4: Revenue forecast chart (full width)  
        [{"colspan": 3}, None, None]
    ],
    subplot_titles=[
        "Current Month Sales", "Next Month Forecast Sales", "Sales Growth",
        "Current Month Revenue", "Next Month Forecast Revenue", "Revenue Growth", 
        "Ferry Sales Forecast", 
        "Ferry Revenue Forecast - Monthly Data (Ticket Price: $8 CAD)"
    ],
    vertical_spacing=0.08
)

# Row 1: Sales KPIs (exactly as you had them)
# Current Month Sales KPI
fig.add_trace(go.Indicator(
    mode="number+delta",
    value=kpis['this_month_sales'],
    number={'suffix': " tickets", 'font': {'size': 30}},
    delta={'reference': kpis.get('prev_month_sales', kpis['this_month_sales']), 'relative': True, 'position': "top"},
    title={"text": kpis.get('this_month_label', 'Current Month')},
), row=1, col=1)

# Next Month Forecast Sales KPI
fig.add_trace(go.Indicator(
    mode="number+delta",
    value=kpis['next_month_sales'],
    number={'suffix': " tickets", 'font': {'size': 30}},
    delta={'reference': kpis['this_month_sales'], 'relative': True, 'position': "top"},
    title={"text": "Next Month Forecast"},
), row=1, col=2)

# Sales Growth Gauge (exactly as you had it)
fig.add_trace(go.Indicator(
    mode="number+gauge",
    value=kpis.get('sales_growth', 0),
    number={'suffix': "%", 'font': {'size': 30}},
    gauge={'axis': {'range': [-50, 50]},
           'bar': {'color': "green" if kpis.get('sales_growth', 0) > 0 else "red"},
           'steps': [{'range': [-50, 0], 'color': "lightgray"},
                    {'range': [0, 50], 'color': "lightgreen"}],
           'threshold': {'line': {'color': "red", 'width': 4},
                        'thickness': 0.75, 'value': 0}},
    title={"text": "Month-over-Month Sales Growth"},
), row=1, col=3)

# Row 2: Revenue KPIs (exactly as you had them)
# Current Month Revenue KPI
fig.add_trace(go.Indicator(
    mode="number+delta",
    value=kpis['this_month_revenue'],
    number={'prefix': "$", 'font': {'size': 30}, 'valueformat': ',.0f'},
    delta={'reference': kpis.get('prev_month_revenue', kpis['this_month_revenue']), 'relative': True, 'position': "top"},
    title={"text": kpis.get('this_month_label', 'Current Month Revenue')},
), row=2, col=1)

# Next Month Forecast Revenue KPI
fig.add_trace(go.Indicator(
    mode="number+delta",
    value=kpis['next_month_revenue'],
    number={'prefix': "$", 'font': {'size': 30}, 'valueformat': ',.0f'},
    delta={'reference': kpis['this_month_revenue'], 'relative': True, 'position': "top"},
    title={"text": "Next Month Forecast Revenue"},
), row=2, col=2)

# Revenue Growth Gauge (exactly as you had it)
fig.add_trace(go.Indicator(
    mode="number+gauge",
    value=kpis.get('revenue_growth', 0),
    number={'suffix': "%", 'font': {'size': 30}},
    gauge={'axis': {'range': [-50, 50]},
           'bar': {'color': "green" if kpis.get('revenue_growth', 0) > 0 else "red"},
           'steps': [{'range': [-50, 0], 'color': "lightgray"},
                    {'range': [0, 50], 'color': "lightgreen"}],
           'threshold': {'line': {'color': "red", 'width': 4},
                        'thickness': 0.75, 'value': 0}},
    title={"text": "Month-over-Month Revenue Growth"},
), row=2, col=3)

# Row 3: Your Sales Forecast (exactly as you had it)
fig.add_trace(go.Scatter(
    x=df['Date'],
    y=df['Sales Count'],
    mode='lines+markers+text',
    text=df['Sales Count'],
    textposition='top center',
    textfont=dict(size=9, color='blue'),
    name='Actual Sales',
    line=dict(color='blue', width=2, shape='spline', smoothing=1.3),
    marker=dict(size=6, color='blue')
), row=3, col=1)

# Create seamless connection for forecast
last_actual_date = df['Date'].iloc[-1]
last_actual_value = df['Sales Count'].iloc[-1]
forecast_dates = [last_actual_date] + list(forecast['ds'])
forecast_values = [last_actual_value] + list(forecast['yhat'])

fig.add_trace(go.Scatter(
    x=forecast_dates,
    y=forecast_values,
    mode='lines+markers+text',
    text=[''] + [f"{int(val)}" for val in forecast['yhat']],
    textposition='top center',
    textfont=dict(size=9, color='red'),
    name='Forecast',
    line=dict(color='red', width=2, dash='dot', shape='spline', smoothing=1.3),
    marker=dict(size=6, color='red')
), row=3, col=1)

# Row 4: Your Revenue Forecast (exactly as you had it)
fig.add_trace(go.Scatter(
    x=df_revenue['Date'],
    y=df_revenue['Revenue'],
    mode='lines+markers+text',
    text=[f"${int(val):,}" for val in df_revenue['Revenue']],
    textposition='top center',
    textfont=dict(size=9, color='blue'),
    name='Actual Revenue',
    line=dict(color='blue', width=2, shape='spline', smoothing=1.3),
    marker=dict(size=6, color='blue')
), row=4, col=1)

# Forecast revenue
last_actual_revenue = df_revenue['Revenue'].iloc[-1]
forecast_revenue_dates = [last_actual_date] + list(forecast_revenue['ds'])
forecast_revenue_values = [last_actual_revenue] + list(forecast_revenue['Revenue'])

fig.add_trace(go.Scatter(
    x=forecast_revenue_dates,
    y=forecast_revenue_values,
    mode='lines+markers+text',
    text=[''] + [f"${int(val):,}" for val in forecast_revenue['Revenue']],
    textposition='top center',
    textfont=dict(size=9, color='red'),
    name='Forecast Revenue',
    line=dict(color='red', width=2, dash='dot', shape='spline', smoothing=1.3),
    marker=dict(size=6, color='red')
), row=4, col=1)

# Update layout to match your original styling
fig.update_layout(
    title={
        'text': '<b>Ferry Sales & Revenue Executive Dashboard</b><br><sub>Complete Analytics & KPI Suite</sub>',
        'x': 0.5,
        'y': 0.98,
        'font': {'size': 24, 'family': 'Arial Black', 'color': '#2c3e50'}
    },
    height=1400,
    width=1600,
    showlegend=True,
    paper_bgcolor='#f8f9fa',
    plot_bgcolor='white',
    font=dict(family="Arial", size=11, color='#2c3e50'),
    legend=dict(
        orientation="h",
        yanchor="top",
        y=-0.05,
        xanchor="center",
        x=0.5,
        bgcolor='rgba(255,255,255,0.9)',
        bordercolor='gray',
        borderwidth=1
    ),
    margin=dict(t=100, b=80, l=60, r=60)  # Add more bottom margin for legend
)

# Update axes to match your original formatting
fig.update_yaxes(title_text="Sales Count", tickformat=',.0f', row=3, col=1)
fig.update_yaxes(title_text="Revenue (CAD)", tickformat='$,.0f', row=4, col=1)
fig.update_xaxes(title_text="Date", row=3, col=1)
fig.update_xaxes(title_text="Date", row=4, col=1)

fig.show()

# Your original summary
print("\n" + "="*70)
print("FERRY OPERATIONS - EXECUTIVE KPI SUMMARY")
print("="*70)
print(f"📊 CURRENT PERFORMANCE:")
print(f"   Current Month Sales: {kpis['this_month_sales']:,} tickets")
print(f"   Current Month Revenue: ${kpis['this_month_revenue']:,.0f} CAD")
print(f"   Month-over-Month Growth: {kpis.get('sales_growth', 0):+.1f}%")
print()
print(f"🔮 NEXT MONTH FORECAST:")
print(f"   Forecasted Sales: {kpis['next_month_sales']:,} tickets")
print(f"   Forecasted Revenue: ${kpis['next_month_revenue']:,.0f} CAD")
print(f"   Expected Change: {kpis['forecast_sales_change']:+.1f}% sales, {kpis['forecast_revenue_change']:+.1f}% revenue")
print()
print(f"💰 KEY METRICS:")
print(f"   Average Ticket Price: ${ticket_price:.2f} CAD")
print(f"   Revenue per Ticket: ${kpis['this_month_revenue']/kpis['this_month_sales']:.2f} CAD")
print("="*70)


FERRY OPERATIONS - EXECUTIVE KPI SUMMARY
📊 CURRENT PERFORMANCE:
   Current Month Sales: 199,215 tickets
   Current Month Revenue: $1,593,720 CAD
   Month-over-Month Growth: -46.2%

🔮 NEXT MONTH FORECAST:
   Forecasted Sales: 211,682 tickets
   Forecasted Revenue: $1,693,459 CAD
   Expected Change: +6.3% sales, +6.3% revenue

💰 KEY METRICS:
   Average Ticket Price: $8.00 CAD
   Revenue per Ticket: $8.00 CAD
